# Trading Bot Testing Notebook
This notebook outlines the testing process for the trading bot components, including data management, model training, prediction, strategy, and execution.

## Setup
Import necessary modules and set up environment variables.

In [23]:
# Import necessary modules
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import numpy as np

from data_manager import get_historical_data, preprocess_data
from model import create_model, train_model, predict
from strategy import define_trade_action
from executor import execute_trade, check_open_positions
from config import BASE_URL, API_KEY, SECRET_KEY

# Set up any necessary environment variables or configurations here

## 1. Daily Data Fetching and Processing

In [24]:
symbol = 'AAPL'
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')  # Adjust based on model needs

historical_data = get_historical_data(symbol, start_date, end_date)
historical_data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-01-24,195.419998,196.380005,194.339996,194.500000,194.252121,53631300
2024-01-25,195.220001,196.270004,193.110001,194.169998,193.922546,54822100
2024-01-26,194.270004,194.759995,191.940002,192.419998,192.174774,44594000
2024-01-29,192.009995,192.199997,189.580002,191.729996,191.485657,47145600
2024-01-30,190.940002,191.800003,187.470001,188.039993,187.800354,55859400
2024-01-31,187.039993,187.100006,184.350006,184.399994,184.164993,55467800
2024-02-01,183.990005,186.949997,183.820007,186.860001,186.621872,64885400
2024-02-02,179.860001,187.330002,179.250000,185.850006,185.613159,102518000
2024-02-05,188.149994,189.250000,185.839996,187.679993,187.440811,69668800


## Model Training and Prediction Testing
Test the model's training and prediction capabilities.

In [32]:
# Assuming df is your DataFrame with the stock data

# Select features and target
feature_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
features = historical_data[['Open', 'High', 'Low', 'Close', 'Volume']]
target = historical_data['Close'].shift(-1)  # Predicting the next day's close price

# Scale features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Prepare sequences for LSTM
seq_length = 10  # Number of days to look back
train_data = []
train_labels = []

for i in range(len(features_scaled) - seq_length):
    train_data.append(features_scaled[i:i+seq_length])
    train_labels.append(target.iloc[i+seq_length])

train_data = np.array(train_data)
train_labels = np.array(train_labels)

# Now, train_data and train_labels are ready for model training


In [15]:
# Example test for model training
model = create_model(input_shape=(None, 5))  # Adjust based on your model's input shape
history = train_model(model, train_data, train_labels, batch_size=32, epochs=10)
print(history.history)

Epoch 1/10


/Users/pablodigiusto/miniforge3/envs/trading_bot/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: nan
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: nan
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: nan
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nan
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: nan
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nan
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: nan
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nan
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: nan
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: nan
{'loss': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]}


## Strategy Testing
Test the trading strategy's decision-making process based on simulated predictions.

In [33]:
# Example test for strategy
predictions = predict(model, preprocess_data(historical_data,feature_columns))
actions = define_trade_action(predictions, threshold=0.01)
print(actions)

ValueError: Unrecognized data type: x=(array([[1.        , 1.        , 1.        , 1.        , 0.20581747],
       [0.98714672, 0.99211466, 0.91848933, 0.97449754, 0.22516245],
       [0.92609293, 0.88387033, 0.84095464, 0.839258  , 0.05900318],
       [0.78084806, 0.70035812, 0.6845596 , 0.7859348 , 0.10045487],
       [0.71208249, 0.67168474, 0.54473182, 0.50077237, 0.24201378],
       [0.46143919, 0.33476769, 0.33797265, 0.21947417, 0.23565209],
       [0.26542452, 0.32401436, 0.30285013, 0.40958285, 0.38864449],
       [0.        , 0.35125483, 0.        , 0.33153074, 1.        ],
       [0.532776  , 0.48888899, 0.43671292, 0.47295162, 0.46635259],
       [0.44987154, 0.49318988, 0.49834368, 0.5981456 , 0.04108129],
       [0.69280209, 0.61792135, 0.62027852, 0.60664642, 0.20269348],
       [0.61246788, 0.50967702, 0.53677986, 0.52241178, 0.        ],
       [0.56490968, 0.54193593, 0.57985435, 0.56336995, 0.06812009],
       [0.55012846, 0.44731183, 0.49966833, 0.43199345, 0.013319

## Execution Testing
Simulate trade execution based on strategy decisions. Initially, use mock or paper trading.

In [6]:
# Example test for execution
# Note: Use mock or paper trading API for testing
symbol = 'AAPL'
quantity = 1
for action in actions:
    execute_trade(symbol, quantity, action)

## Integration Testing
Perform end-to-end tests to ensure all components work together seamlessly.

In [ ]:
# Example integration test
# Run the full process from data fetching, model training, prediction, strategy decision, to execution.
# This is a comprehensive test to ensure the entire system functions as expected.